<br><br>
## <center>Lab 2 - Twitter:</center>##
#### <center>The easiest entry point for doing "big data" style research</center> ####
<br><br>
There is a reason why you've seen and heard about so much research based on Twitter data. Because it aims to be a place for public discussion and dissemination of information, Twitter makes its easy to algorithmically collect its data, including metadata and other things not rendered on in feeds. This comparative ease with which anyone can get the data make it a go-to source for trying to answer some types of questions. The jury is still out on how generalizable the finding are, but President Trump's affinity for the platform has kept it relevant and interesting.
<br><br>
In this lab you will explore Twitter data and produce some descriptive findings. This module can gather, archive, search and display tweets interactively. Twitter does not typically provide access to all tweets, but instead allows users to find particular tweets or sample from realtime or historical data. Twitter claims samples are random but it's sampling schemes aren't public and you should always remember that any inferences you make should come with the caveat that you don't know anything about the sampling procedure that produced the sample.
<br><br>
### Using the Twitter API ###
<br>
Twitter provides developers (that's us!) an *Application Programming Interface (API)* for interacting with their servers. It provides many functions for both collecting existing data and posting new data.  Like most APIs, it uses the [REST protocol](https://spring.io/understanding/REST), which means if you know the right way to format the requests you can send them and get responses using just your web browser. It turns out this is clunky and inconvenient so people create libraries mapping commands written in your favorite programming language to the proper REST requests. We'll be making use of one of these below so you need to have it on your computer. We'll also need the package/library `whoosh` for creating a searchable index. Use the Anaconda Navigator or the command line to install the packages. The Navigator tabs for environments you can search for the packages by name and import them. Use either `easy_install twitter` or `pip3 install twitter` for command line commands.

Next we need to import the core module with the `import twttr` statement below. We'll also import a package for displaying HTML within this notebook using the `from IPython.dispay import HTML` statement.



In [ ]:
from tools import twttr
from IPython.display import HTML

<br>

Before you can get started with collecting and analyzing, you'll need to authenticate to the Twitter API. The very first time you run the code in the cell below, you'll need to authenticate using your own Twitter account, so if you don't have one you'll want to register now. 

When you hit shift+enter to run the code the first time, you'll be taken to the Twitter website to sign in. It will then provide you with a PIN number to insert into the newly visible blue highlighted space in the cell above. Once you've done that, it will save your login data. You'll still need to run the login code everytime you start this notebook, but you won't need to authenticate via Twitter's webpage again.


In [ ]:
my_connection = twttr.TWTTR()
my_connection.login()

<br>
### Getting Data Through the API ###

Now we're all authenticated to the Twitter API and get start getting data. You probably don't want to just start getting the realtime feed, so let's start with something just to whet our appetite. Let's just find a user to start.
The command `twttr.viewUser("user_handle")` does that and reports some of the user's metadata. It also returns a string that allows you to render the results as HTML using the aptly named `HTML()` function.

In [ ]:
a = my_connection.viewUser("chrisrock")

In [ ]:
HTML(a)

The `HTML` command is not at all important right here but we'll use it later to make some of the more complex outputs easier to read.

The `viewUser` command can be helpful to make sure you have the right person, but you'll probably stick to other, more powerful commands to actually get meaningful data.
<br><br>
## getUsersTweets( ) ##
One such command is `getUsersTweets()`.
It gets up to 200 of the user's most recent tweets. Below is an example of its use to get 25 tweets from ESPN's NBA account.<br><br>

In [ ]:
the_tweets = my_connection.getUsersTweets("ESPNNBA", count=25)

The results of this request contained a lot more than you see above. The tweets have been cleaned and reformated to make it easier and more enjoyable to quickly see what might be relevant to your research.

<br>
## index_Tweets()##

After you've run the `getUsersTweets()` command, all of the cleaned Tweet objects are sitting in the list `the_tweets`. You can preview them in the output window beneath the command after you've run it, but if you want to save them for future analysis, you must index them using the `index_Tweets( )` command. This command creates a searchable database using the Python module `whoosh`. We'll cover how to do searchs later in this lab, but first we need to get them into the database so that we have something to search through!
#### Important Note ####
The seearchable index is saved in the folder named `twitter_index`, which is in the same folder as this notebook. Saving it allows the index to be maintained if you close this notebook or session. Once you start indexing the first time, all new indexing will add to the same index. If you'd like to start over, you'll need to clear and reset the index. The command for that is below. All entries will be lost so be careful!<br>
`my_connection.clear_index()`<br><br>

In [ ]:
my_connection.index_Tweets(the_tweets)

<br>
This command does a lot of things behind the scenes, including running the text through the *Linguistic Inquiry and Word Count* package to create *sentiment* scores. We'll cover that topic later.<br>

An important thing to know is that this command accepts a `batch` argument to expedite retrieving the collection of tweets you just added to the index. The index pools everything you put into it so the batch label is important if you want to track the different results from your queries. To use this functionality, simply provide a unique batch number whenever you run the command. You can then provide this number as an index search term later. Otherwise the default batch number is 1 and does not automatically increment, meaning all tweets will be in batch 1. I highly recommend noting the contents of the batches in somewhere such as one of the cells in this notebook. <br><br>


In [ ]:
my_connection.index_Tweets(the_tweets, batch=2)

## tweetSearch(   ) ##
Searching by users can be a great way to get data, but you'll probably also want to search by terms, hashtags, or even URLs. To do that, you use the `tweetSearch( )` method shown here. Twitter caps the number of tweets it will return at 100 which you might expect is a very small sample of all the possible tweets matching your search term(s). Twitter doesn't tell us much about how they determine the sample, but they do allow us to specify whether we want the most recent tweets, the most popular or a mixture of both. You can specify this with the second argument below; the options are `popular`, `recent`, or `mixed` and the default if you omit this argument is `recent`.

In [ ]:
movieTweets = my_connection.tweetSearch("Last Jedi", "recent")

Again, to ensure that you can look at and analyze these tweets later, you need to index them using the `index_Tweets( )` command.

In [ ]:
my_connection.index_Tweets(movieTweets, batch=2)

<br>
## Search the index ##
<br>
The `tweetSearch( )` command searches Twitter for tweets related to the term you supplied, but there will unexplored context in those results. To explore that context, we index the tweets (metadata and all) and then search within our database. You use the `search( )` command to do this. `search( )` accepts arguments for any combination of fields stored in the index. For this particular index, those terms are as follows:

  + `tweet_id`        - The unique integer that identifies this tweet
  + `owner_name`      - The displayed name of the original poster (i.e. Chris Rock)
  + `owner_sn`        - The screen name of the original poster (i.e. chrisrock)
  + `owner_id`        - The Twitter ID of the original poster (i.e. 238319766)
  + `batch`           - The batch number that you've been specifying. (default=1)
  + `text`            - The content of the tweet itself
  + `posted`          - The time the tweet was posted (GMT +0:00)
  + `isRT`            - (True/False) Whether the tweet was a retweet
  + `timesRT`         - The number of times the tweet was retweeted
  + `timesFav`        - The number of times the tweet was favorited
  + `hashtags`        - All the hashtags appearing in the tweet
  + `mentions`        - All the mentions in the tweet
  + `media`           - The included media type, if applicable
  
These arguments are entered all together in a single string, a series of character within double quotation marks. We'll go over the details of the syntax shortly.
  
In addition to these index arguments, you can search by a period of the day using the regular `time_slice` argument. The argument looks like this: `["09:00":"17:00"]`. The period starts at the time on the left and ends on the time on the right and it is important that there are two numbers on each side of the colon. If the starting time is later than the ending time, it searches past midnight and into the next day.

You can also limit the number of tweets returned using the regular `limit` argument. The default is 100.

Finally, you can save your search results for viewing as an HTML page later using the `saveAs` argument. The default name is `search`
<br>
Below are some examples of proper syntax. All of these variants can be combined into a single command but are in separate commands here for visual clarity.

<br><br><br>
Here is an example looking at the hashtags and posting times. We'll address the data the command returns later in the lab.

In [ ]:
results, data, display, tweet_list = my_connection.search("hashtags:starwars posted:'Jan 1 to Jan 17'")

Note the use of multiple quotation marks. **Any argument in that appears in the bulleted list above must appear within the double quotation marks**. There are also single quotation marks for the values that follow the key terms. Also note that there is no comma between key terms

In [ ]:
results, data, display, tweet_list = my_connection.search("'women' isRT:'True'", saveAs="StarWars")

The *timeRT* and *timesFav* key terms allow you to find the most popular tweets (or the least or in between).  Whether there is an important difference between favoriting or retweeting from the prospective of users is not clear to me, but you have both at your disposal. <br><br>
Here are additional examples of search queries.


In [ ]:
results, data, display, tweet_list = my_connection.search("content:'women' mentions:'@starwars'", limit=5)

This next one gets up to 100 tweets made between the hours of 10pm and 3am and save the results in a file named LateNight_tweets.html in the folder this notebook is in.

In [ ]:
results, data, display, tweet_list = my_connection.search("batch:'2' posted:'Jan 1 to Jan 17'", time_slice=["22:00","03:00"], saveAs="LateNight_tweets")

The query below searches the text of the tweets for the variations of a phrase.

In [ ]:
results, data, display, tweet_list = my_connection.search("'Star Wars' OR 'starwars' OR 'StarWars' OR '#starwars'",saveAs="StarWars")

This next one searches for tweets that are either tagged with a search term or mentions a user's offical Twitter account.

In [ ]:
results, data, display, tweet_list = my_connection.search("hashtags:'starwars' OR mentions:'@starwars'")

#### Remember #### 
All of these search terms can be used simultaneously. Once you get the hang of it, you can do some very powerful sorting. The syntax can be finicky so if you get 0 results and you're expecting some, make sure there aren't extra commas or spaces and that the quotation marks are always in pairs.

#### What search( ) does####
Running the search command does a lot of things behind the scenes. It packages information up for you in the terms that appear on the left hand side of the equals sign. 
The first term is named `results` but you can name it whatever you'd like by replacing `results` with your term before running the command. (You can do this for all terms by the way.) Results is an object that can be used to display the averages of the individual LIWC values for the whole group of Tweets. 
`data` is the raw information that can be used to find differences between LIWC averages from different search results, a topic covers below. 
`display` can be used to view the individual tweets in the search results in a HTML table. 
Finally, `tweet_list` is a list of the Tweet IDs for the tweets in the search results.
<br>
<br>
The other thing that the command does is produce an HTML file with the Tweets represented as circles proportional to the number of times it was retweeted. An example of the link can be found [here.](files/tools/tweetpack.html) Your search results show up in the folder this notebook is in under the name you specified with the `saveAs` argument.

<br><br>
## Inspecting Retweets##
What you've been collecting and looking at up until now are just a handful of tweets but you might really want to look at what happens to the tweet after the original poster submitted it. Because of Twitter's rate-limiting policies, this can be a time consuming task and accordingly the indexing command doesn't collect those data immediately. Rather, you need to tell the index which ones to collect using the `inspectRetweets( )` command. The argument is just a list of tweet ids, most likely gotten from the `search( )` command above.

In [ ]:
ind = min(5,len(tweet_list))

my_connection.inspectRetweets(tweet_list[:ind])

This command can take a long time to run and the amount of time depends on two things. The first is the number of tweets; the more you have in the list argument, the longer it will take. The second is whether or not the tweets are from the same person. The code looks up the poster's friends and followers to determine relationship the person has to the retweeters and saves the data. This process is slow (in the world of computers, that is) so if you are looking at posts from a bunch a people, it can take a long time. So if you have a stack of tweets from Oprah, it might take a couple of minutes. But if you have the most popular Tweets about the latest Star Wars movie, you could get dinner and it might not be finished.
<br><br>
### What Inspecting Retweets Does###
Inspecting the retweets looks at the who retweeted the original, when they did and the nature of their relationship to the original poster. It also summarizes these data and visualizes the network of the retweets in a webpage reachable by clicking on the tweet in search interface. The circle representing the tweet will turn blue if you've inspected it.  You can then click on it and be taken to the page with more details. 

<br>
This page includes a visualization of the network in which the length of the ties is proportional to the number of hours after the original tweet that the retweet was posted. The nodes are also colored according to the retweeter's relationship with the original poster. An important thing to note about these data is that Twitter limits the information to a random sampling of 100 of the retweets. So while the tweet might have 2000 retweets, you'll see at most 101 nodes in this network. For the purpose of this lab, you may consider this a representative sample and conduct your analyze using these data.

## Backtracking: Viewing Results ##

When you run a search, you can see the titles of the posts that the query found but it is actually spitting back much more information. We can look at those using the data object `results`, `data`, and `display`.
<br>
While it appears last, let's start with `display` first. It contains all of the raw information from the posting returned by the query. To make it easier to look at, it is packaged up as an HTML file that we can look at by running the command below.

In [ ]:
HTML(display)

A couple of things to notice about this:

1) That is a lot of information in our poor little Notebook. If you'd rather not look at it here, don't run the command and just go find the file named 'search_results.html' in your CL folder. You can even just keep that file open in your browser and refresh it after new searchs. Doing so will make your notebook easier to use and the HTML also renders better there.

2) There is big table at the bottom of each post. These contain the textual analysis of the post using the LIWC dictionary. It contains basic statistics like word counts (WC) and the number of pronouns but also the frequency of words related to positive emotions (posemo) and tentativeness (tentat). The full names and categories for the terms are available on the LIWC site [here](http://www.liwc.net/descriptiontable1.php)

3) The tables aren't the same for each post. That's because some posted scored zero on the LIWC count and they've been omitted in order to condense the information. 

4) These tables are really good for exploring patterns worth looking into more, but chances are you aren't going to want to find averages for these tables by hand. Thats what the `results` object does for you.<br>

## Viewing Group Statistics##
The *results* object contains useful information about that group of search results, namely the average, the standard deviation, the maximum value and the minimum value for each of the LIWC categories. You can display it as HTML using the command below or via the file *search_averages.html*

In [ ]:
HTML(results)

Finally, the `data` object contains the raw numbers of the results. These can be used to compare different search groups and create smaller tables of more relevant data. To do that, you need to specify two different searches and give the outputs different names as is shown in the next two lines

In [ ]:
results_1, data_1, display_1, tweet_list_1 = my_connection.search("batch:2 content:'women'", saveAs="morning")

In [ ]:
results_2, data_2, display_2, tweet_list_2 = my_connection.search("batch:2 content:'men'", saveAs="afternoon")

You then put the two data objects, 'data_1' and 'data_2' into the 'LIWC_differences( )' command to get a table back. You can then display it by passing it to the 'HTML( )' command. The two lines below do it for the searches above

In [ ]:
diffs = my_connection.LIWC_differences(data_1, data_2)

In [ ]:
HTML(diffs)

Finally, if you'd like to trim this big table to just the relevant information, run the 'LIWC_differences_subset( )' command with a list of the terms you want. This is a handy thing for producing actual results to be included somewhere. 

In [ ]:
subset = my_connection.LIWC_differences_subset(data_1, data_2, ['WC', 'posemo', 'negemo', 'swear', 'anger', 'affect'])

In [ ]:
HTML(subset)